# GFF/GTF to four-column conversion for QUAST

Our input data is in non-SO gene model-compliant GTF/GFF. Quast requires `gene`/`operon` features, or a four column tab-separated format with:

* `seqname\tID\tstart\tend`

which is nice and general, and implies one contiguous region per gene. It avoids many of the incompatibilities between GFF and GTF, and we can interpret the features of interest at the exon and gene levels separately.

The input data is in `Phyca11_filtered_genes.gff`. We'll read it in with `csv`.

## Only keeping exons

Exons don't have unique IDs in the input file. But, as we don't really care which genes they come from (at the moment) we can assign them unique arbitrary IDs.

In [ ]:
import csv
with open("Phyca11_filtered_genes.gff") as fh:
    with open("Phyca11_filtered_exons_quast.tab", "w") as outfh:
        reader = csv.reader(fh, delimiter='\t', quotechar='"')
        writer = csv.writer(outfh, delimiter='\t')
        for idx, row in enumerate([row for row in reader
                                   if row[2] == 'exon']):
            seqname = row[0]
            start, end = int(row[3]), int(row[4])
            strand = row[6]
            if strand == '-':
                start, end = end, start
            exonid = "%06d" % (idx + 1)
            writer.writerow([seqname, exonid, start, end])

## Keeping genes/CDS

We can keep a 'gene' from the GFF/GTF file by looking at the contiguous sequence between start and stop codons only (not just the exons).

This code makes my teeth itch - we're making *many* assumptions about good behaviour of the input data which are not justified. For instance, `gw1.1.971.1` only has a start codon, not a stop codon in the input file. There may be worse problems.

Using start and stop codons may have inaccuracies, but at least we 'know' there's only one start and one stop codon (most of the time). To know how many exons/CDS exist per gene, we would need to read the whole file in first.

In [ ]:
import csv
with open("Phyca11_filtered_genes.gff") as fh:
    with open("Phyca11_filtered_genes_quast.tab", "w") as outfh:
        reader = csv.reader(fh, delimiter='\t', quotechar='"')
        writer = csv.writer(outfh, delimiter='\t')
        genedict = {}
        count = 0
        for idx, row in enumerate([row for row in reader
                                   if row[2] in ('start_codon',
                                                'stop_codon')]):
            #print(idx, row)
            seqname = row[0]
            start, end = int(row[3]), int(row[4])
            strand = row[6]
            genename = row[8].split(';')[0][5:].replace('"','')
            if genename not in genedict:
                genedict[genename] = [start, end]
            else:
                genedict[genename] += [start, end]
                count += 1
                geneid = "%06d" % count
                minval, maxval = min(genedict[genename]), max(genedict[genename])
                if strand == '-':
                    minval, maxval = maxval, minval
                writer.writerow([seqname, geneid, minval, maxval])
                #print(seqname, geneid, minval, maxval, strand, genename, genedict[genename])
            #if count > 10:
            #    break